In [1]:
import resp2

import logging as log

Failed to import openforcefield and/or OpenEye toolkit.


In [2]:
log.getLogger().setLevel(log.INFO)
resp2.create_target(name ='ethanol',density=789.3,hov = 42.3, dielectric=32.7,smiles='CCO',resname ='ETH')

INFO:root:Created target file ethanol-liquid/data.csv
    Density: 789.3 g/l
    Heat of Vaporization: 42.3 kcal/mol
    Dielectric Constant: 32.7
INFO:root:The following SMILES string will be converted: CCO
INFO:root:-=# Output #=- Created ethanol-liquid/ETH.mol2 containing single molecule
INFO:root:-=# Output #=- Created ethanol-liquid/ETH.pdb containing single molecule
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.053 seconds
INFO:root:Running gmx insert-molecules to create a solvent box...
INFO:root:Time elapsed:  0.838 seconds
INFO:root:-=# Output #=- Created ethanol-liquid/ETH-box.pdb containing solvent box with 700 molecules and length 4.267


0

In [3]:
number_of_conformers=resp2.create_conformers(infile='ethanol-liquid/ETH.mol2', outfile='ethanol-liquid/ETH-conformers.mol2')




INFO:root:Created conformations for ethanol-liquid/ETH.mol2 and saved them to ethanol-liquid/ETH-conformers.mol2


In [4]:
resp2.optimize_conformers(name ='ethanol',resname ='ETH', opt = True,number_of_conformers=number_of_conformers)
resp2.create_respyte(name ='ethanol',resname ='ETH', type='RESP2LIQUID',number_of_conformers=number_of_conformers)
resp2.create_respyte(name ='ethanol',resname ='ETH', type='RESP2GAS',number_of_conformers=number_of_conformers)
resp2.create_respyte(name ='ethanol',resname ='ETH', type='RESP1',number_of_conformers=number_of_conformers)
resp2.create_charge_file(name='ethanol',resname ='ETH', type = 'RESP1', delta= 2.0)
resp2.create_charge_file(name='ethanol',resname ='ETH', type = 'RESP2', delta= 0.5)

INFO:root:Optimization of ethanol and conformer 1 succesful
INFO:root:Optimization of ethanol and conformer 2 succesful
INFO:root:Create folder structure for ethanol with 2 conformers


TypeError: create_respyte_input_files() got an unexpected keyword argument 'resname'